In [2]:
# Importación de librerías
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
# Estructura del modelo
"""
Endógenas:
    - cantidad_ventas
    - valor_ventas
    
Exógenas:

    # Tiempo
    - periodo_year -> Año
    - periodo_month_number -> Mes
    
    # Geográficas
    - geo_ciudad -> Clasificación de Zona [1 => Bogotá D.C., 2 => Cali]
    - geo_clase_zona -> Clasificación de Zona [1 => Residencial, 2 => Mixta, 3 => Industrial, 4 => Comercial]
    - geo_seguridad -> Nivel de seguridad [1 => Deficiente, 2 => Medio, 3 => Bueno]
    - geo_cercania_lugares -> Nivel de cercanía a lugares de interés [1 => Deficiente, 2 => Medio, 3 => Bueno]
    - geo_calidad_infraestructura -> Nivel de calidad de infraestructuras [1 => Deficiente, 2 => Medio, 3 => Bueno]
    
    # Proyecto
    - perfil_comprador -> Clasificación del perfil del comprador [1 => Vivienda, 2 => Vivienda alto valor, 3 => Inversión]
    - perfil_comprador_complementario -> Clasificación del perfil del comprador [1 => Vivienda, 2 => Vivienda alto valor, 3 => Inversión]
    - clase_proyecto -> Clase de proyecto [0 => VIS, 1 => No VIS]
    - precio_promedio_unidad -> Precio promedio proyecto
    - precio_min_unidad -> Precio mínimo unidad
    - precio_max_unidad -> Precio máximo unidad
    - plazo_promedio_unidad -> Plazo en meses (c/i) promedio proyecto
    - plazo_min_unidad -> Plazo en meses (c/i) mínimo unidad
    - plazo_max_unidad -> Plazo en meses (c/i) máximo unidad
    - area_promedio_unidad -> Área promedio unidad
    - area_min_unidad -> Área mínima unidad
    - area_max_unidad -> Área máxima unidad
    - area_proyecto -> Área proyecto
    - area_zonas_comunes -> Área zonas comunes
    - calidad_unidades -> Calidad de acabados
    - presupuesto_marketing -> Presupuesto marketing
    
    # Macroeconómicos
    - macro_pib_const -> PIB trimestral a precios constantes
    - macro_ipc -> IPC trimestral
    - macro_consumo -> Índice de expectativas de consumo (Camacol - DANE)
    - macro_calidad_cartera -> Índice de calidad de cartera
    - macro_valor_subsidios -> Valor de subsidios desembolsados
    - macro_expectativa_subsidios -> Expectativa de desembolso de subsidios
    - macro_confianza_consumidor -> Índice de confianza del consumidor
    - macro_incertidumbre_bancaria -> Índice de incertidumbre bancario
    - macro_indice_precios_vivienda -> Índice de precios de la vivienda
    - macro_tasa_interes -> Tasa de interés
    - macro_tasa_interes_hipoteca -> Tasa de interés mercado hipotecario
"""

In [8]:
# Cargar datos
datos = pd.read_excel('data/macroproyectos.xlsx')
datos.head()

C:\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Año,Mes,MacNombreMacro,empresa_x,geo_ciudad,geo_clase_zona,geo_seguridad,geo_cercania_lugares,geo_calidad_infraestructura,perfil_comprador,...,plazo_promedio_unidad,plazo_min_unidad,plazo_max_unidad,area_promedio_unidad,area_min_unidad,area_max_unidad,area_proyecto,calidad_unidades,presupuesto_marketing,cantidad_ventas
0,2023,9,CONJUNTO RESIDENCIAL PUERTO TRANQUILO,PATRIMONIOS AUTONOMOS FIDUCIARIA DAVIVIENDA,RICAURTE,NaN,NaN,NaN,NaN,Vivienda alto valor,...,224.000000,64,369,78.293650,0.00,206.10,57702.42,NaN,NaN,NaN
1,2023,9,LA ALEGRIA I DE LA MARLENE,CUSEZAR PATRIMONIO AUTONOMO - FIDUDAVIVIENDA,BOGOTÁ D.C.,NaN,NaN,NaN,NaN,Vivienda,...,111.000000,111,111,38.765833,29.93,39.15,21398.74,NaN,NaN,NaN
2,2023,9,SABANA DE LOS CEREZOS,URBE PATRIMONIO AUTONOMO FIDUDAVIVIENDA,BOGOTÁ D.C.,NaN,NaN,NaN,NaN,Vivienda,...,3.000000,3,3,66.800000,35.00,79.90,22044.00,NaN,NaN,NaN
3,2023,9,LA UNION II DE LA MARLENE,CUSEZAR PATRIMONIO AUTONOMO - FIDUDAVIVIENDA,BOGOTÁ D.C.,NaN,NaN,NaN,NaN,Vivienda,...,409.333333,277,551,45.765908,31.20,53.00,30754.69,NaN,NaN,NaN
4,2023,9,LA ALEGRIA III DE LA MARLENE,CUSEZAR PATRIMONIO AUTONOMO - FIDUDAVIVIENDA,BOGOTÁ D.C.,NaN,NaN,NaN,NaN,Vivienda,...,277.333333,247,308,38.765833,29.93,39.15,26050.64,NaN,NaN,NaN


In [19]:
# Limpieza de datos
macroproyectos = datos.copy()

# Remoción de variables
columnas_a_remover = [
    'MacNombreMacro'
]

# Si tienen sufijo '_x'
macroproyectos = macroproyectos[
    [x for x in macroproyectos.columns if (not str(x).endswith('_x')) and (x not in columnas_a_remover)]].copy()

# Remoción columnas vacías
macroproyectos.dropna(inplace=True, axis=1, how='all')

# Rellenado de faltantes en columnas no nulas
macroproyectos.fillna(0, inplace=True)

macroproyectos.head()

,Año,Mes,geo_ciudad,perfil_comprador,perfil_comprador_complementario,clase_proyecto,precio_promedio_unidad,precio_min_unidad,precio_max_unidad,plazo_promedio_unidad,plazo_min_unidad,plazo_max_unidad,area_promedio_unidad,area_min_unidad,area_max_unidad,area_proyecto,cantidad_ventas
0,2023,9,RICAURTE,Vivienda alto valor,Vivienda alto valor,0,3.169162e+08,0,1077790000,224.000000,64,369,78.293650,0.00,206.10,57702.42,0.0
1,2023,9,BOGOTÁ D.C.,Vivienda,Inversión,1,1.159672e+08,87954600,117432600,111.000000,111,111,38.765833,29.93,39.15,21398.74,0.0
2,2023,9,BOGOTÁ D.C.,Vivienda,Vivienda,0,2.788751e+08,158660000,397890000,3.000000,3,3,66.800000,35.00,79.90,22044.00,0.0
3,2023,9,BOGOTÁ D.C.,Vivienda,Inversión,1,1.803223e+08,124950000,200940000,409.333333,277,551,45.765908,31.20,53.00,30754.69,0.0
4,2023,9,BOGOTÁ D.C.,Vivienda,Inversión,1,1.161496e+08,89760000,117096000,277.333333,247,308,38.765833,29.93,39.15,26050.64,0.0


In [20]:
# Categorización
variables_categoricas = [
    'geo_ciudad',
    'perfil_comprador',
    'perfil_comprador_complementario'
]

# Se obtienen variables dummy
macroproyectos = pd.get_dummies(macroproyectos, columns=variables_categoricas, drop_first=True)

macroproyectos.head()

,Año,Mes,clase_proyecto,precio_promedio_unidad,precio_min_unidad,precio_max_unidad,plazo_promedio_unidad,plazo_min_unidad,plazo_max_unidad,area_promedio_unidad,...,area_max_unidad,area_proyecto,cantidad_ventas,geo_ciudad_CALI,geo_ciudad_LA CALERA,geo_ciudad_RICAURTE,perfil_comprador_Vivienda,perfil_comprador_Vivienda alto valor,perfil_comprador_complementario_Vivienda,perfil_comprador_complementario_Vivienda alto valor
0,2023,9,0,3.169162e+08,0,1077790000,224.000000,64,369,78.293650,...,206.10,57702.42,0.0,0,0,1,0,1,0,1
1,2023,9,1,1.159672e+08,87954600,117432600,111.000000,111,111,38.765833,...,39.15,21398.74,0.0,0,0,0,1,0,0,0
2,2023,9,0,2.788751e+08,158660000,397890000,3.000000,3,3,66.800000,...,79.90,22044.00,0.0,0,0,0,1,0,1,0
3,2023,9,1,1.803223e+08,124950000,200940000,409.333333,277,551,45.765908,...,53.00,30754.69,0.0,0,0,0,1,0,0,0
4,2023,9,1,1.161496e+08,89760000,117096000,277.333333,247,308,38.765833,...,39.15,26050.64,0.0,0,0,0,1,0,0,0


In [27]:
# Definición del modelo
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

modelo = LinearRegression()

data_x = macroproyectos[[x for x in macroproyectos.columns if x != 'cantidad_ventas']]
data_y = macroproyectos['cantidad_ventas']

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.33, random_state=42)

modelo.fit(X_train, y_train)

# Resultados
print('Los estimadores (\u03B2_x) son: ', modelo.coef_)
print('El intercepto (\u03B2_0) es: ', modelo.intercept_)

# Métricas
print('El coeficiente de determinación (R²) es: ', modelo.score(X_train, y_train))
y_predicted = modelo.predict(X_test)
print('El error cuadrático medio (RMSE) es: ', mean_squared_error(y_test, y_predicted, squared=False))

Los estimadores (β_x) son:  [ 3.29552457e-01  1.06847703e-01 -1.39035460e+01  1.66758057e-08
  8.58691666e-08 -1.76652730e-08  4.96594918e-02 -2.22027763e-02
 -2.89897162e-02 -9.83962457e-01 -4.17842094e-01  3.98842005e-01
  9.04356956e-05  1.38025492e+01  6.14152781e+00 -2.87790769e+01
  4.22686975e+00 -1.17130493e+00 -4.56146827e+00 -1.17130493e+00]
El intercepto (β_0) es:  -630.9966098843523
El coeficiente de determinación (R²) es:  0.12063684804772146
El error cuadrático medio (RMSE) es:  9.015418546485419
